In [1]:
%matplotlib inline
import os
import re
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
from skimage import measure, io
import matplotlib.pyplot as plt
import mahotas as mh
import matplotlib.mlab as mlab
import matplotlib.ticker as ticker

/home/ws/ih7618/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
def parse_filename(filepath):
    basename, ext = os.path.splitext(os.path.basename(filepath))

    comps = basename.split('_')
    size = tuple([int(v) for v in comps[-1:][0].split('x')])
    bits = int(re.findall('\d+', comps[-2:-1][0])[0])
    name = '_'.join(comps[:-2])

    return name, bits, size, ext

In [3]:
def segment_data(filepath):
    data = open_data(filepath)
    # apply 3D median filter 2x2x2
    # calculate Reyni thresholding
    # crop

In [4]:
def open_data(filepath):
    _, glob_ext = os.path.splitext(os.path.basename(filepath))
    data = None

    if glob_ext == '.raw':
        name, bits, size, ext = parse_filename(filepath)
        data_type = np.float32 if bits == 32 else np.uint8
        data = np.memmap(filepath, dtype=data_type, shape=tuple(reversed(size)), mode='r')
    elif glob_ext == '.nii.gz' or glob_ext == '.nii' or glob_ext == '.gz':
        data = nib.load(filepath).get_data()
    elif glob_ext == '.tif':
        data = io.imread(filepath)
    else:
        print 'Incorrent file format, or filename.'

    return data

In [5]:
def save_fig(fig, output_filepath):
    output_path, filename = os.path.split(output_filepath)
        
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    print output_filepath
    
    plt.tight_layout()
    fig.savefig(output_filepath)

In [6]:
def write_as_raw(data, sample_name, output_dir, prefix=None):
    bits = -1
    if data.dtype == np.int32 or data.dtype == np.float32:
        bits = 32
    elif data.dtype == np.uint8 or data.dtype == np.bool:
        bits = 8

    size = data.shape[::-1]
    output_filename = '{0}_{1}bit_{2}x{3}x{4}.raw'.format(sample_name, bits, *size) if prefix is None \
                        else '{0}_{1}_{2}bit_{3}x{4}x{5}.raw'.format(sample_name, prefix, bits, *size)
    data.tofile(os.path.join(output_dir, output_filename))


In [7]:
_MEASUREMENTS = {
    'Label': 'label',
    'Area': 'area',
    'Perimeter': 'perimeter'
}

_MEASUREMENTS_VALS = _MEASUREMENTS.values()

def object_counter(stack_binary_data):
    print 'Object counting - Labeling...'
    labeled_stack, num_labels = ndi.measurements.label(stack_binary_data)
    objects_stats = pd.DataFrame(columns=_MEASUREMENTS_VALS)
    
    print 'Object counting - Stats gathering...'
    for slice_idx in np.arange(labeled_stack.shape[0]):
        for region in measure.regionprops(labeled_stack[slice_idx]):
            objects_stats = objects_stats.append({_measure: region[_measure] \
                                        for _measure in _MEASUREMENTS_VALS}, \
                                            ignore_index=True)
            
    print 'Object counting - Stats grouping...'
    objects_stats = objects_stats.groupby('label', as_index=False).sum()

    return objects_stats, labeled_stack

In [8]:
def estimate_particles(samples, output_dir, parts=None):
    for sample_name, params in samples['samples'].items():
        if 'particles' in params:
            print '###### Processing of {}'.format(sample_name)

            print 'Data processing - Opening...'
            input_data = open_data(os.path.join(samples['input_dir'], params['particles'][0]))
        
            if parts is not None:
                input_data = split_dataset(input_data, parts=parts)
                
                for i, dp in enumerate(input_data):
                    dp_objects_stats, dp_labeled_data = object_counter(dp)
                    output_path = os.path.join(output_dir, sample_name, 'part_{}'.format(i))
                    
                    print 'Data storing (Part #{}) - Stats and data saving...'.format(i)
                    if not os.path.exists(output_path):
                        os.makedirs(output_path)
                        
                    dp_objects_stats.to_csv(os.path.join(output_path, 'particles_stats_{}_part{}.csv'.format(sample_name, i)))
            else:
                objects_stats, labeled_data = object_counter(input_data)
                output_path = os.path.join(output_dir, sample_name)

                print 'Data storing - Stats and data saving...'
                if not os.path.exists(output_path):
                    os.makedirs(output_path)

                objects_stats.to_csv(os.path.join(output_path, 'particles_stats_{}.csv'.format(sample_name)))

In [45]:
def estimate_porosity(samples, output_dir=None, parts=None):
    for sample_name, sample_info in samples['samples'].items():
        print '###### Processing of {}'.format(sample_name)
        
        if parts is not None:
            total_volume_dp, material_volume_dp = [0]*(parts[0]*parts[1]), [0]*(parts[0]*parts[1])
            
            for structure_type, data_params in sample_info.items():
                input_data = open_data(os.path.join(samples['input_dir'], data_params[0]))
                input_data = split_dataset(input_data, parts=parts)
                
                for i, dp in enumerate(input_data):
                    if not total_volume_dp[i]:
                        total_volume_dp[i] = dp.size
                        
                    material_volume_dp[i] += np.count_nonzero(dp)
                    
            for i in xrange(len(total_volume_dp)):
                output_path = os.path.join(output_dir, sample_name, 'part_{}'.format(i))
                
                print 'Data storing - Stats and data saving (Part #{})...'.format(i)
                
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                    
                mt = material_volume_dp[i] / float(total_volume_dp[i])
                porosity = 1. - mt
                print 'Porosity :{}'.format(porosity)
                
                np.savetxt(os.path.join(output_path, 'porosity_{}_part{}.txt'.format(sample_name, i)), [porosity])
        else:
            total_volume, material_volume = None, 0
        
            for structure_type, data_params in sample_info.items():
                print 'Data processing - Opening...'
            
                input_data = open_data(os.path.join(samples['input_dir'], data_params[0]))
                
                if data_params[3] is not None:
                    input_data = input_data[data_params[3]]
            
                if total_volume is None:
                    total_volume = input_data.size
                
                material_volume += np.count_nonzero(input_data)
            
            if output_dir is not None:
                output_path = os.path.join(output_dir, sample_name)
            
                print 'Data storing - Stats and data saving...'
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
            
            mt = material_volume / float(total_volume)
            porosity = 1. - mt
            print 'Porosity: {0}'.format(porosity)
            
            if output_dir is not None:
                np.savetxt(os.path.join(output_path, 'porosity_{0}.txt'.format(sample_name)), [porosity])

In [25]:
def estimate_codirectionality(samples, output_dir, parts=None):
    for sample_name, params in samples['samples'].items():
        print '###### Processing of {}'.format(sample_name)
        
        data = np.load(os.path.join(samples['input_dir'], params['fibers'][0])).item()
        
        for key in params['fibers'][1]:
            input_data = data[key]
            input_data = np.rad2deg(input_data)
        
            print 'Data processing - Opening...'
            input_data = split_dataset(input_data, parts=parts)
                
            for i, dp in enumerate(input_data):
                dp = dp[np.where(dp > 0)]

                output_path = os.path.join(output_dir, sample_name, 'part_{}'.format(i))
                    
                print 'Data storing (Part #{}) - Stats and data saving...'.format(i)
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                
                pd.DataFrame({'orientation': dp}).to_csv(os.path.join(output_path, 'orientation_{}_{}_part{}.csv').format(sample_name, key, i))

In [26]:
def estimate_diameter(samples, output_dir, parts=None):
    for sample_name, params in samples['samples'].items():
        print '###### Processing of {}'.format(sample_name)
        
        input_data = np.load(os.path.join(samples['input_dir'], params['fibers'][0])).item()[params['fibers'][1]]
        input_data = np.rad2deg(input_data)
        
        print 'Data processing - Opening...'
        input_data = split_dataset(input_data, parts=parts)
                
        for i, dp in enumerate(input_data):
            dp = dp.ravel()

            output_path = os.path.join(output_dir, sample_name, 'part_{}'.format(i))
                    
            print 'Data storing (Part #{}) - Stats and data saving...'.format(i)
            if not os.path.exists(output_path):
                os.makedirs(output_path)
                
            pd.DataFrame({'diameter': dp}).to_csv(os.path.join(output_path, 'diameter_{}_part{}.csv').format(sample_name, i))

In [27]:
def split_dataset(data, parts=(3,3)):
    y_idxs = np.array_split(np.arange(data.shape[1]), 3)
    x_idxs = np.array_split(np.arange(data.shape[2]), 3)
    
    out_datasets = []
    
    for i in xrange(parts[0]):
        for j in xrange(parts[1]):
            out_datasets.append(data[:, slice(y_idxs[i][0],y_idxs[i][-1]), slice(x_idxs[j][0],x_idxs[j][-1])])
            
    return out_datasets

In [38]:
config_samples = {
    'input_dir': './raw_polymer_data/cropped',
    'samples': {
        'PCL_SiHA_cl': {
            'particles': ('PCL9%_Si-HA10%_cl/analysis/particles_PCL9%_Si-HA10%_cl_32bit_800x800x351.tif', np.uint8, (351,800,800), None),
            'fibers': ('PCL9%_Si-HA10%_cl/analysis/fibers_PCL9%_Si-HA10%_cl_8bit_800x800x351.raw', np.uint8, (351,800,800), None)
        }
        ,
        'PCL_SiHA_wa': {
            'particles':('PCL9%_Si-HA10%_wa/analysis/particles_PCL9%_Si-HA10%_wa_32bit_800x800x351.tif', np.uint8, (351,800,800), None),
            'fibers': ('PCL9%_Si-HA10%_wa/analysis/fibers_PCL9%_Si-HA10%_wa_8bit_800x800x351.raw', np.uint8, (351,800,800), None)
        },
        'PCL_cl': { #30-90
            'fibers': ('PCL_9%_cl/analysis/fiber_mask_PCL_cl_8bit_800x800x221.raw', np.uint8, (221,800,800), (30,60))
        },
        'PCL_wa': {#35-170
            'fibers': ('PCL_9%_wa/analysis/fiber_mask_PCL_9%_wa_8bit_800x800x251.raw', np.uint8, (251,800,800), (35,170)),
        }
    }
}

In [75]:
config_samples_new = {
    'input_dir': './raw_polymer_data/cropped',
    'samples': {
        'PCL_SiHA_cl': {
            'particles': ('PCL9%_Si-HA10%_cl/analysis/fibers_PCL9%_Si-HA10%_cl_32bit_800x800x484.tif', np.uint8, (484,800,800), None)
        },
        'PCL_SiHA_wa': {
            'particles':('PCL9%_Si-HA10%_wa/analysis/fibers_PCL9%_Si-HA10%_wa_8bit_800x800x351.raw', np.uint8, (351,800,800), None)
        },
        'PCL_cl': { #30-90
            'fibers': ('PCL_9%_cl/analysis/fiber_mask_porosity2_PCL_cl_8bit_800x800x121.raw', np.uint8, (91,800,800), None)
        },
        'PCL_wa': {#35-170
            'fibers': ('PCL_9%_wa/analysis/fiber_mask_porosity_PCL_9%_wa_8bit_800x800x141.raw', np.uint8, (141,800,800), None),
        }
    }
}

In [29]:
config_orient_samples = {
    'input_dir': './raw_polymer_data/results',
    'samples': {
        'PCL_SiHA_cl': {
            'fibers': ('PCL_SiHA_cl_w31_orientation_evaluation.npy', ('azth', 'lat'))
        },
        'PCL_SiHA_wa': {
            'fibers': ('PCL_SiHA_wa_w31_orientation_evaluation.npy', ('azth', 'lat'))
        },
        'PCL_cl': {
            'fibers': ('PCL_cl_w31_orientation_evaluation.npy', ('azth', 'lat'))
        },
        'PCL_wa': {
            'fibers': ('PCL_wa_w31_orientation_evaluation.npy', ('azth', 'lat')),
        }
    }
}

In [30]:
config_diameter_samples = {
    'input_dir': './raw_polymer_data/results',
    'samples': {
        'PCL_SiHA_cl': {
            'fibers': ('PCL_SiHA_cl_diameter_evaluation.npy', ('diameter'))
        },
        'PCL_SiHA_wa': {
            'fibers': ('PCL_SiHA_wa_diameter_evaluation.npy', ('diameter'))
        },
        'PCL_cl': {
            'fibers': ('PCL_cl_diameter_evaluation.npy', ('diameter'))
        },
        'PCL_wa': {
            'fibers': ('PCL_wa_diameter_evaluation.npy', ('diameter'))
        }
    }
}

# Estimate particles

In [20]:
estimate_particles(config_samples, './data/results', parts=None)

###### Processing of PCL_SiHA_cl
Data processing - Opening...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing - Stats and data saving...


In [72]:
estimate_particles(config_samples, './data/results', parts=(3,3))

###### Processing of PCL_SiHA_cl
Data processing - Opening...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #0) - Stats and data saving...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #1) - Stats and data saving...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #2) - Stats and data saving...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #3) - Stats and data saving...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #4) - Stats and data saving...
Object counting - Labeling...
Object counting - Stats gathering...
Object counting - Stats grouping...
Data storing (Part #5) - Stats and data saving...
Object counting - La

# Estimate porosity

In [21]:
estimate_porosity(config_samples, output_dir='./data/results')

###### Processing of PCL_cl
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.83875627121
###### Processing of PCL_SiHA_cl
Data processing - Opening...
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.740096416489
###### Processing of PCL_SiHA_wa
Data processing - Opening...
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.731141706731
###### Processing of PCL_wa
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.761474962649


In [76]:
estimate_porosity(config_samples_new, output_dir='./data/results')

###### Processing of PCL_cl
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.771454907025
###### Processing of PCL_SiHA_cl
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.783606898889
###### Processing of PCL_SiHA_wa
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.735002581909
###### Processing of PCL_wa
Data processing - Opening...
Data storing - Stats and data saving...
Porosity: 0.722717198582


# Split orientation datasets

In [30]:
estimate_codirectionality(config_orient_samples, './data/results', parts=(3,3))

###### Processing of PCL_cl
Data processing - Opening...
Data storing (Part #0) - Stats and data saving...
Data storing (Part #1) - Stats and data saving...
Data storing (Part #2) - Stats and data saving...
Data storing (Part #3) - Stats and data saving...
Data storing (Part #4) - Stats and data saving...
Data storing (Part #5) - Stats and data saving...
Data storing (Part #6) - Stats and data saving...
Data storing (Part #7) - Stats and data saving...
Data storing (Part #8) - Stats and data saving...
Data processing - Opening...
Data storing (Part #0) - Stats and data saving...
Data storing (Part #1) - Stats and data saving...
Data storing (Part #2) - Stats and data saving...
Data storing (Part #3) - Stats and data saving...
Data storing (Part #4) - Stats and data saving...
Data storing (Part #5) - Stats and data saving...
Data storing (Part #6) - Stats and data saving...
Data storing (Part #7) - Stats and data saving...
Data storing (Part #8) - Stats and data saving...
###### Process

# Split diameter dataset

In [27]:
estimate_diameter(config_diameter_samples, './data/results', parts=(3,3))

###### Processing of PCL_cl
Data processing - Opening...
Data storing (Part #0) - Stats and data saving...
Data storing (Part #1) - Stats and data saving...
Data storing (Part #2) - Stats and data saving...
Data storing (Part #3) - Stats and data saving...
Data storing (Part #4) - Stats and data saving...
Data storing (Part #5) - Stats and data saving...
Data storing (Part #6) - Stats and data saving...
Data storing (Part #7) - Stats and data saving...
Data storing (Part #8) - Stats and data saving...
###### Processing of PCL_SiHA_cl
Data processing - Opening...
Data storing (Part #0) - Stats and data saving...
Data storing (Part #1) - Stats and data saving...
Data storing (Part #2) - Stats and data saving...
Data storing (Part #3) - Stats and data saving...
Data storing (Part #4) - Stats and data saving...
Data storing (Part #5) - Stats and data saving...
Data storing (Part #6) - Stats and data saving...
Data storing (Part #7) - Stats and data saving...
Data storing (Part #8) - Stats 